# 🐺 INSIDER CONVICTION HUNTER
## Finding Code P Transactions in Market Movers

**Mission:** Scan top gainers/losers from past week and validate insider conviction (Code P buys)

**Strategy:**
1. Get top movers (gainers AND losers - wounded prey strategy)
2. Run Form 4 validator on each ticker
3. Find stocks with recent Code P insider buying
4. Filter for conviction scores
5. Identify next IONQ/RGTI/AISP opportunities

**Wolf Pack Tools:** Form4Validator + Multi-Signal System

**AWOOOO** 🐺

In [ ]:
# Import required libraries
import sys
import os
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path
sys.path.append('/workspaces/trading-companion-2026/src')
sys.path.append('/workspaces/trading-companion-2026/src/research')

# Import Form 4 validator
from research.form4_validator import Form4Validator

print("✅ Libraries imported")
print(f"📅 Current date: {datetime.now().strftime('%Y-%m-%d')}")

## 📈 Step 1: Get Top Movers from Past Week

We'll scan stocks from multiple sources:
- Top gainers (momentum plays)
- Top losers (wounded prey strategy)
- High volume movers

In [ ]:
# Configuration
SCAN_PERIOD = 7  # Last 7 days
MIN_PRICE = 1.0  # Filter penny stocks
MAX_PRICE = 50.0  # Focus on small/mid caps
MIN_VOLUME = 100000  # Minimum daily volume

# Build candidate list - start with known movers + sectors
# You can add more tickers from finviz/stocktwits/etc.
candidate_tickers = [
    # AI/Quantum (hot sector)
    'AISP', 'SOUN', 'IONQ', 'RGTI', 'QBTS', 'ARQQ',
    # Space/Defense
    'LUNR', 'RKLB', 'PL', 'ASTS', 'SPIR',
    # Small cap tech
    'BBAI', 'SMBR', 'SMCI', 'NVDA', 'ARM', 'AVGO',
    # Biotech movers
    'RXRX', 'SDGR', 'FATE', 'CRSP', 'BEAM',
    # Recent IPOs
    'RDDT', 'KVUE', 'ARM', 'BIRK',
    # EV/Clean energy
    'RIVN', 'LCID', 'CHPT', 'PLUG', 'FCEL',
    # Cybersecurity
    'CRWD', 'S', 'PANW', 'ZS',
    # Other momentum
    'HOOD', 'COIN', 'MARA', 'RIOT', 'TSLA'
]

print(f"🎯 Scanning {len(candidate_tickers)} tickers for weekly performance...")
print(f"📅 Period: Last {SCAN_PERIOD} days")
print(f"💰 Price range: ${MIN_PRICE} - ${MAX_PRICE}")

In [ ]:
# Fetch weekly performance data
def get_weekly_performance(tickers, days=7):
    """Get price change over last N days"""
    results = []
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days+5)  # Extra buffer for market days
    
    for ticker in tickers:
        try:
            # Download data
            stock = yf.Ticker(ticker)
            hist = stock.history(start=start_date, end=end_date)
            
            if len(hist) < 2:
                continue
                
            # Calculate metrics
            start_price = hist['Close'].iloc[0]
            end_price = hist['Close'].iloc[-1]
            current_price = end_price
            pct_change = ((end_price - start_price) / start_price) * 100
            avg_volume = hist['Volume'].mean()
            
            # Filter criteria
            if current_price < MIN_PRICE or current_price > MAX_PRICE:
                continue
            if avg_volume < MIN_VOLUME:
                continue
                
            results.append({
                'ticker': ticker,
                'start_price': round(start_price, 2),
                'current_price': round(current_price, 2),
                'pct_change': round(pct_change, 2),
                'avg_volume': int(avg_volume),
                'days': len(hist)
            })
            
        except Exception as e:
            print(f"⚠️  {ticker}: {str(e)[:50]}")
            continue
    
    return pd.DataFrame(results)

# Run the scan
print("🔍 Fetching market data...")
df = get_weekly_performance(candidate_tickers, SCAN_PERIOD)
print(f"✅ Got data for {len(df)} tickers")

# Display summary
if len(df) > 0:
    df_sorted = df.sort_values('pct_change', ascending=False)
    print(f"\n📊 Performance Range: {df['pct_change'].min():.1f}% to {df['pct_change'].max():.1f}%")
    df_sorted.head(10)

## 🔍 Step 2: Identify Top Gainers and Losers

Split into two hunting groups:
- **Top Gainers**: Momentum continuation plays
- **Top Losers**: Wounded prey with insider buying = turnaround signal

In [ ]:
# Split into gainers and losers
TOP_N = 15  # Take top 15 from each category

gainers = df.nlargest(TOP_N, 'pct_change')
losers = df.nsmallest(TOP_N, 'pct_change')

print("🟢 TOP GAINERS (Past Week)")
print("="*60)
for idx, row in gainers.iterrows():
    print(f"{row['ticker']:6s} | +{row['pct_change']:6.1f}% | ${row['current_price']:6.2f} | Vol: {row['avg_volume']:,}")

print("\n🔴 TOP LOSERS (Past Week)")
print("="*60)
for idx, row in losers.iterrows():
    print(f"{row['ticker']:6s} | {row['pct_change']:7.1f}% | ${row['current_price']:6.2f} | Vol: {row['avg_volume']:,}")

# Combine for insider analysis
scan_tickers = list(gainers['ticker']) + list(losers['ticker'])
print(f"\n🎯 Total tickers to scan for insider buying: {len(scan_tickers)}")

## 🐺 Step 3: Run Form 4 Validator - Find Code P Transactions

Now the hunt begins - validate insider conviction on each ticker.

**What we're looking for:**
- Code P = Open market purchases (REAL conviction)
- Recent transactions (last 90 days)
- Multiple insiders buying = stronger signal
- Large dollar amounts = serious conviction

**This is where we find the next 10x plays.** 🎯

In [ ]:
# Initialize Form 4 Validator
validator = Form4Validator(debug=False)

# Scan function with error handling
def scan_insider_buying(ticker, days=90):
    """Scan for Code P transactions in last N days"""
    try:
        result = validator.validate_ticker(ticker, days)
        return result
    except Exception as e:
        return None

# Run insider scan on all movers
print("🔍 Scanning for insider buying (Code P transactions)...")
print("⏱️  This will take a few minutes - SEC rate limits to 10 req/sec\n")

insider_results = []

for i, ticker in enumerate(scan_tickers):
    print(f"[{i+1}/{len(scan_tickers)}] Scanning {ticker}...", end=' ')
    
    result = scan_insider_buying(ticker, days=90)
    
    if result and result['conviction_count'] > 0:
        # Found Code P transactions!
        print(f"✅ FOUND {result['conviction_count']} Code P buys (${result['total_conviction']:,.0f})")
        
        # Get price data for this ticker
        ticker_data = df[df['ticker'] == ticker].iloc[0] if ticker in df['ticker'].values else None
        
        insider_results.append({
            'ticker': ticker,
            'pct_change': ticker_data['pct_change'] if ticker_data is not None else 0,
            'current_price': ticker_data['current_price'] if ticker_data is not None else 0,
            'code_p_count': result['conviction_count'],
            'total_conviction_value': result['total_conviction'],
            'conviction_score': result['conviction_score'],
            'latest_buy_date': result.get('latest_conviction_date', 'Unknown')
        })
    else:
        print("❌ No Code P")

print(f"\n✅ Scan complete! Found {len(insider_results)} tickers with insider buying")

## 📊 Step 4: Analyze Results - The Hunt List

**Pack hunting strategy:**
- Gainers + insider buying = Momentum with conviction
- Losers + insider buying = Wounded prey turnaround signal (IONQ pattern!)
- Sort by conviction score for priority

In [ ]:
# Create results DataFrame
if len(insider_results) > 0:
    results_df = pd.DataFrame(insider_results)
    results_df = results_df.sort_values('conviction_score', ascending=False)
    
    print("🎯 WOLF PACK HUNT LIST - Stocks with Insider Buying")
    print("="*80)
    print(f"{'Ticker':<8} {'Price':>8} {'Wk Chg':>8} {'P Buys':>8} {'$ Value':>12} {'Score':>8} {'Latest':<12}")
    print("="*80)
    
    for idx, row in results_df.iterrows():
        ticker = row['ticker']
        price = row['current_price']
        pct = row['pct_change']
        buys = row['code_p_count']
        value = row['total_conviction_value']
        score = row['conviction_score']
        date = row['latest_buy_date']
        
        # Color code by performance
        direction = "🟢" if pct > 0 else "🔴"
        
        print(f"{direction} {ticker:<6} ${price:7.2f} {pct:+7.1f}% {buys:>8} ${value:>11,.0f} {score:>8}/100 {date:<12}")
    
    print("="*80)
    print(f"\n✅ Total opportunities: {len(results_df)}")
    
    # Categorize
    gainers_with_insider = results_df[results_df['pct_change'] > 0]
    losers_with_insider = results_df[results_df['pct_change'] < 0]
    
    print(f"\n🟢 Gainers with insider buying: {len(gainers_with_insider)} (momentum + conviction)")
    print(f"🔴 Losers with insider buying: {len(losers_with_insider)} (wounded prey turnaround signal)")
    
else:
    print("❌ No tickers found with Code P insider buying in the scan list")
    print("💡 Try expanding the candidate ticker list or date range")

## 🔬 Step 5: Deep Dive on Top Candidates

Pick the top 3-5 by conviction score and dig deeper:
- Run full multi-signal analysis
- Check technical setup
- Validate thesis

Run individual cells below for detailed analysis.

In [ ]:
# Deep dive on specific ticker
def analyze_ticker_details(ticker):
    """Get detailed insider transaction breakdown"""
    print(f"\n{'='*80}")
    print(f"🔍 DEEP DIVE: {ticker}")
    print(f"{'='*80}\n")
    
    # Run full validation with details
    validator_detail = Form4Validator(debug=False)
    result = validator_detail.validate_ticker(ticker, days=90)
    
    if not result or result['conviction_count'] == 0:
        print(f"❌ No Code P transactions found for {ticker}")
        return
    
    print(f"✅ Found {result['conviction_count']} Code P conviction buys")
    print(f"💰 Total conviction value: ${result['total_conviction']:,.0f}")
    print(f"📊 Conviction score: {result['conviction_score']}/100\n")
    
    # Show transactions
    if 'transactions' in result and result['transactions']:
        print("📋 TRANSACTION DETAILS:")
        print("-" * 80)
        for trans in result['transactions']:
            if trans['code'] == 'P':  # Code P only
                print(f"📅 {trans['date']}")
                print(f"   {trans['name']} ({trans['title']})")
                print(f"   {trans['shares']:,} shares @ {trans['price']}")
                if trans['value'] > 0:
                    print(f"   💰 Value: ${trans['value']:,.0f}")
                print()

# Example: Analyze top result
if len(insider_results) > 0:
    top_ticker = insider_results[0]['ticker']
    print(f"🎯 Analyzing top conviction ticker: {top_ticker}")
    analyze_ticker_details(top_ticker)
else:
    print("💡 Run the scanner first to get results")

## 💾 Step 6: Export Results

Save the hunt list for further analysis and tracking.

In [ ]:
# Export results to CSV
if len(insider_results) > 0:
    output_file = f'/workspaces/trading-companion-2026/logs/insider_hunt_{datetime.now().strftime("%Y%m%d")}.csv'
    results_df.to_csv(output_file, index=False)
    print(f"✅ Results saved to: {output_file}")
    print(f"📊 Total opportunities exported: {len(results_df)}")
    
    # Summary stats
    print(f"\n📈 SUMMARY STATISTICS:")
    print(f"   Average conviction score: {results_df['conviction_score'].mean():.1f}/100")
    print(f"   Total conviction capital: ${results_df['total_conviction_value'].sum():,.0f}")
    print(f"   Average weekly performance: {results_df['pct_change'].mean():+.1f}%")
    
    # Top 5 by conviction
    print(f"\n🏆 TOP 5 BY CONVICTION SCORE:")
    for idx, row in results_df.head(5).iterrows():
        print(f"   {row['ticker']}: {row['conviction_score']}/100 | ${row['total_conviction_value']:,.0f} insider buying")
else:
    print("❌ No results to export")

## 🎯 Next Steps

**To expand the hunt:**

1. **Add more tickers**: Expand `candidate_tickers` list with tickers from:
   - Finviz screeners
   - StockTwits trending
   - Twitter/Reddit momentum
   - Sector-specific lists

2. **Run multi-signal analysis**: Use the multi_signal_scanner.py on top results

3. **Manual validation**: Check SEC filings, news, technicals for final picks

4. **Track results**: Monitor performance of insider buying signals over time

**AWOOOO - Happy hunting!** 🐺